# B⁺ Yield Ratio Stability Analysis (Blocks)

Perform temporal stability analysis of B⁺ decay yields across different data-taking periods to monitor detector and reconstruction performance.

## Analysis Overview

• **Purpose**: Monitor systematic variations in B⁺ → D⁰π⁺ / B⁺ → J/ψK⁺ yield ratios across blocks
• **Method**: Extract fitted yields from `fit_results` trees and calculate ratios with propagated uncertainties  
• **Statistical Test**: Constant fit to ratios tests temporal stability (χ²/ndf goodness-of-fit)
• **Output**: Stability plots, fit parameters, and statistical summary of variations

## Important Notes

• **B2OC Files**: Uses `nsig` branch for B⁺ → D⁰π⁺ signal yield
• **B2CC Files**: Uses `njpsik` branch for B⁺ → J/ψK⁺ signal yield (not total signal)
• **Error Propagation**: Proper quadrature sum for uncertainties in ratios and fits
• **Blocks**: Data organized by time periods (UP/DOWN magnet polarity per block) 


In [ ]:
# Importing required functions
import ROOT as r                           
import numpy as np                         
import matplotlib.pyplot as plt             
from scipy.optimize import curve_fit
from pathlib import Path

DATA_CLEAN = Path("data/processed_clean_bp_p")  # Directory containing processed ROOT files

# Input files for B⁺ → D⁰π⁺ and B⁺ → J/ψK⁺ analysis
b2oc_files = [
    DATA_CLEAN/"2024_B2OC_UP_B5.root",
    DATA_CLEAN/"2024_B2OC_DOWN_B6.root", 
    DATA_CLEAN/"2024_B2OC_DOWN_B7.root",
    DATA_CLEAN/"2024_B2OC_UP_B8.root"
]
b2cc_files = [
    DATA_CLEAN/"2024_B2CC_UP_B5.root",
    DATA_CLEAN/"2024_B2CC_DOWN_B6.root",
    DATA_CLEAN/"2024_B2CC_DOWN_B7.root", 
    DATA_CLEAN/"2024_B2CC_UP_B8.root"
]

blocks = [5, 6, 7, 8]                       

def get_yield_with_error_from_file(filename, yield_branch='nsig', error_branch='nsig_err'):
    """
    Extract yield and error from ROOT file fit_results tree
    For B2OC files: uses 'nsig' and 'nsig_err' branches  
    For B2CC files: uses 'njpsik' and 'njpsik_err' branches for J/ψK yield
    """
    try:
        file = r.TFile.Open(str(filename))   
        if not file or file.IsZombie():
            print(f"Cannot open {filename}")
            return 0, 0
            
        # Access fit_results tree containing fitted parameters
        tree = file.Get("fit_results")
        if not tree:
            print(f"fit_results tree not found in {filename}")
            file.Close()
            return 0, 0
        
        # Extract yield and error values from specified branches
        yield_values = []
        error_values = []
        
        for i in range(tree.GetEntries()):
            tree.GetEntry(i)
            yield_val = getattr(tree, yield_branch, 0)
            error_val = getattr(tree, error_branch, 0)
            yield_values.append(yield_val)
            error_values.append(error_val)
        
        # Sum yields and propagate uncertainties
        total_yield = sum(yield_values)
        total_error = np.sqrt(sum(err**2 for err in error_values))  
            
        file.Close()
        return total_yield, total_error
            
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return 0, 0

# Extract yields for each block with proper branch names
print("Extracting yields from block files...")
b2oc_yields = []
b2oc_errors = []
b2cc_yields = []
b2cc_errors = []

for i, block in enumerate(blocks):
    print(f"\nBlock {block}:")
    
    # B⁺ → D⁰π⁺ yield and error (uses nsig branch)
    b2oc_yield, b2oc_error = get_yield_with_error_from_file(b2oc_files[i], 'nsig', 'nsig_err')
    b2oc_yields.append(b2oc_yield)
    b2oc_errors.append(b2oc_error)
    print(f"  B2OC yield: {b2oc_yield:.2f} ± {b2oc_error:.2f}")
    
    # B⁺ → J/ψK⁺ yield and error (uses njpsik branch for J/ψK signal)
    b2cc_yield, b2cc_error = get_yield_with_error_from_file(b2cc_files[i], 'njpsik', 'njpsik_err')
    b2cc_yields.append(b2cc_yield)
    b2cc_errors.append(b2cc_error)
    print(f"  B2CC yield: {b2cc_yield:.2f} ± {b2cc_error:.2f}")

# Calculate yield ratios with uncertainty propagation
b2oc_yields = np.array(b2oc_yields)
b2oc_errors = np.array(b2oc_errors)
b2cc_yields = np.array(b2cc_yields)
b2cc_errors = np.array(b2cc_errors)

ratios = b2oc_yields / b2cc_yields        

# Error propagation for ratios: σ_R = R * sqrt((σ_A/A)² + (σ_B/B)²)
ratio_errors = ratios * np.sqrt((b2oc_errors/b2oc_yields)**2 + (b2cc_errors/b2cc_yields)**2)

print(f"\nYield ratios with propagated uncertainties:")
for i, block in enumerate(blocks):
    print(f"  Block {block}: {ratios[i]:.6f} ± {ratio_errors[i]:.6f}")

# Create ratio stability plot with error bars
plt.figure(figsize=(10, 6))
plt.errorbar(blocks, ratios, yerr=ratio_errors, fmt='o-', color='blue', markersize=8, linewidth=2, 
             label='B2OC/B2CC Ratio', capsize=5, ecolor='blue', capthick=2)
plt.xlabel('Block Number', fontsize=12)
plt.ylabel('Yield Ratio (B2OC/B2CC)', fontsize=12)
plt.title('Stability of B2OC/B2CC Ratio vs Block Number', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(blocks)

# Constant fit to test stability (weighted by uncertainties)
def constant_fit(x, a):
    """Zeroth order polynomial (constant function)"""
    return np.full_like(x, a)

try:
    # Perform weighted fit using ratio uncertainties
    weights = 1.0 / ratio_errors**2    
    popt, pcov = curve_fit(constant_fit, blocks, ratios, sigma=ratio_errors, absolute_sigma=True)
    a_fit = popt[0]
    a_err = np.sqrt(pcov[0,0])
    
    # Plot the fitted constant line
    x_fit = np.linspace(min(blocks)-0.5, max(blocks)+0.5, 100)
    y_fit = constant_fit(x_fit, a_fit)
    plt.plot(x_fit, y_fit, '--', color='red', linewidth=2, 
             label=f'Constant Fit: {a_fit:.4f} ± {a_err:.4f}')
    
    # Calculate χ² with proper weighting
    chi2 = np.sum(((ratios - a_fit) / ratio_errors)**2)
    ndf = len(ratios) - 1                  
    chi2_reduced = chi2 / ndf if ndf > 0 else 0
    
    # Add fit statistics to plot
    plt.text(0.02, 0.98, f'χ²/ndf = {chi2_reduced:.3f}\nConstant = {a_fit:.6f} ± {a_err:.6f}', 
             transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    print(f"\nConstant fit results:")
    print(f"Fitted value: {a_fit:.6f} ± {a_err:.6f}")
    print(f"χ² = {chi2:.3f}")
    print(f"χ²/ndf = {chi2_reduced:.3f}")
    print(f"Degrees of freedom: {ndf}")
    
except Exception as e:
    print(f"Fitting failed: {e}")

plt.legend()
plt.ylim(0.68, 0.74)                      
plt.tight_layout()
plt.show()